# Name: Umad ul hassan Rai
# Exercise 1


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.types import StringType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql import Row
from pyspark.sql.functions import concat, col, lit, split, udf, size
from pyspark.sql import functions as sf
from operator import add
import csv
import time


In [2]:
# initialise Spark Session
sparkSession = SparkSession.builder.appName("Exercise1").getOrCreate()
sc = sparkSession.sparkContext
sqlC = SQLContext(sc)

# Exercise 1.2

In [3]:
# I have measure time for exercise 1.3 which i have put in comments
# Top 10 word count took around 46 mintues to complete 

# Creating userRating RDD from users_libraries.
rdd1 = sparkSession.sparkContext.textFile("users_libraries.txt")
userRatingsRDD = rdd1.map(lambda x: (x.split(';')[0], x.split(';')[1].split(',')))

# Extracting key/value pair from key/values(list) so i can join
exploded = userRatingsRDD.flatMap(lambda l: [(l[0], value) for value in l[1]])

#Reversing rdd to paper_id/User to perform join on papersTermRdd
reversed_ = exploded.map(lambda x: (x[1],x[0]))

userRatingsRDD.take(1)
#reversed_.take(10)
#exploded.take(10)

[('f05bcffe7951de9e5a32fff4a42eb088',
  ['1158654',
   '478707',
   '12054725',
   '6670515',
   '781057',
   '13329754',
   '781055',
   '2825',
   '553840',
   '988447',
   '12938139',
   '12037219',
   '12788583',
   '6595566',
   '4027225',
   '920055',
   '3129258',
   '2242776',
   '3112352',
   '144287',
   '706033',
   '525396',
   '622633',
   '9172127',
   '7357993',
   '230211',
   '12790816',
   '503161',
   '12937120',
   '942241',
   '166220',
   '8493138',
   '2945819',
   '227173',
   '11191048',
   '949352',
   '227096',
   '11597926',
   '921623',
   '833638',
   '882809',
   '12738996',
   '99',
   '7154210',
   '11852474',
   '72879',
   '10723139',
   '11923443',
   '2862386',
   '802634',
   '1959297',
   '4488840',
   '12735311',
   '5102465',
   '5177766',
   '129',
   '1467354',
   '9396734',
   '494336',
   '1218954',
   '2783692',
   '3733678',
   '8423325',
   '1160828',
   '507529',
   '7804574',
   '5201306',
   '407124',
   '504896',
   '3281478',
   '136

In [10]:
# Using csv to parse the papers.CSV file to map paper id and abstract as Key/Value
# Also allows to replace null bytes in csv file.
# Creating paperTermsRDD
rdd2 = sparkSession.sparkContext.textFile("papers.csv")
paperTermsRDD = rdd2.map(lambda line: next(csv.reader(x.replace("\x00", "") for x in [line]))) \
                    .map(lambda fields: (fields[0], fields[14]))
paperTermsRDD.take(1)

[('80546',
  "the genetic code has been regarded as arbitrary in the sense that the codon-amino acid assignments could be different than they actually are. this general idea has been spelled out differently by previous, often rather implicit accounts of arbitrariness. they have drawn on the frozen accident theory, on evolutionary contingency, on alternative causal pathways, and on the absence of direct stereochemical interactions between codons and amino acids. it has also been suggested that the arbitrariness of the genetic code justifies attributing semantic information to macromolecules, notably to {dna}. i argue that these accounts of arbitrariness are unsatisfactory. i propose that the code is arbitrary in the sense of jacques monod's concept of chemical arbitrariness: the genetic code is arbitrary in that any codon requires certain chemical and structural properties to specify a particular amino acid, but these properties are not required in virtue of a principle of chemistry. th

# Exercise 1.3

In [12]:
# BroadCasting StopWords
# Timer
start = time.time()
data = [word.rstrip() for word in open("stopwords_en.txt", 'r').readlines()]
broadcastVar = sc.broadcast(data)
# broadcastVar.value

In [13]:
# Joined UserRatingsRDD(Reversed) with PaperTermsRDD
joinedRDD = reversed_.join(paperTermsRDD)

# joinedRDD.take(20)

user_abs = joinedRDD.map(lambda x: (x[1][0], x[1][1]))

# user_abs.take(20)


In [15]:
# Reduced by key before counting words for each user.
grouped = user_abs.reduceByKey(lambda x,y: x + y)
grouped.take(2)

[('8123c9baf50aa777ed3211b5b6306bd0',
  "{meme} is a tool for discovering motifs in sets of protein or {dna} sequences. this paper describes several extensions to {meme} which increase its ability to find motifs in a totally unsupervised fashion, but which also allow it to benefit when prior knowledge is available. when no background knowledge is asserted. {meme} obtains increased robustness from a method for determining motif widths automatically, and from probabilistic models that allow motifs to be absent in some input sequences. on the other hand, {meme} can exploit prior knowledge about a motif being present in all input sequences, about the length of a motif and whether it is a palindrome, and (using dirichlet mixtures) about expected patterns in individual motif positions. extensive experiments are reported which support the claim that {meme} benefits from, but does not require, background knowledge. the experiments use seven previously studied {dna} and protein sequence familie

In [16]:
#Remove punctuations to make the text clear and can be split on ' '
def lower_clean_str(x):
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x[1].lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, '')
  return x[0], lowercased_str

punc = grouped.map(lower_clean_str)

punc.take(1)


[('8123c9baf50aa777ed3211b5b6306bd0',
  'meme is a tool for discovering motifs in sets of protein or dna sequences this paper describes several extensions to meme which increase its ability to find motifs in a totally unsupervised fashion but which also allow it to benefit when prior knowledge is available when no background knowledge is asserted meme obtains increased robustness from a method for determining motif widths automatically and from probabilistic models that allow motifs to be absent in some input sequences on the other hand meme can exploit prior knowledge about a motif being present in all input sequences about the length of a motif and whether it is a palindrome and using dirichlet mixtures about expected patterns in individual motif positions extensive experiments are reported which support the claim that meme benefits from but does not require background knowledge the experiments use seven previously studied dna and protein sequence families and 75 of the protein famil

In [ ]:
# Creating Combiner, to use it as combineByKey
createCombiner = (lambda el: [el]) 
mergeVal = (lambda aggregated, el: aggregated + [el]) # append to aggregated
mergeComb = (lambda agg1,agg2: agg1 + agg2 )  # append agg1 with agg2

# Counting words and Combining then using key.
new = punc.map(lambda x: [((x[0], a), 1) for a in x[1].split(' ')]).flatMap(lambda l : l).reduceByKey(add)\
        .filter(lambda x: x[0][1] not in broadcastVar.value).sortBy(lambda x: x[1], False)\
        .map(lambda x: (x[0][0], x[0][1])).combineByKey(cComb,mVal,mComb)
rdd_new = new.map(lambda x: (x[0], x[1][0:10]))
rdd_new.take(2)
print("Time: ", end-start )

# 2750.50794 secs

In [ ]:
# Writing RDD to file
rdd_new.coalesce(1).saveAsTextFile("Result")

# Exercise 1.4: Basic Analysis

In [6]:
# No. of Distinct Users
d_users = userRatingsRDD.countByKey()
total_users = sum(y for x, y in d_users.items())
print(total_users)

#28416

# No. of Distinct Items
d_items = paperTermsRDD.countByKey()
total_items = sum(y for x, y in d_items.items())
print(total_items)

#172079

# No. of Ratings
ratings = exploded.countByValue()
total_ratings = sum(y for x,y in ratings.items())
print(total_ratings)

#828481

28416
172079
828481


In [7]:
# Minimum Rating Given by a user: userhash ratings.
ratings = exploded.countByKey()
ratings = ratings.values()

mini = min(ratings)
print("Minimum: ", mini)

#Minimum: 1

maxx = max(ratings)
print("Maximum: ", maxx)
#Maximum: 1922

Minimum:  1
Maximum:  1922


In [8]:
# Average number ratings of users: For it we will use total ratings/distinct_users we calculated earlier.
avg_rating = total_ratings/total_users
print("Average: ", avg_rating)
#Average:  29.155440596846848

#Standard Deviation:
#stdev

Average:  29.155440596846848


In [9]:
# Minimum number of Ratings an item has received.
# CountByValue giving IOPub data rate exceeded Error.
# Therefore using reversed_ rdd and countByKey.
ratings_ = reversed_.countByKey()
ratings_ = ratings_.values()

min_rat = min(ratings_)
print("Minimum: ", min_rat)

#Minimum: 3
max_rat = max(ratings_)
print("Maximum: ", max_rat)

#Maximum: 924

#Average no. of ratings of Items, for This we will use total_ratings/total_items
avg_rat_item = total_ratings/total_items
print("Average: ", avg_rat_item)

#Average: 4.81453867119172

Minimum:  3
Maximum:  924
Average:  4.81453867119172


# Exercise 1.5 DataFrames

In [10]:
# Using two columns as StringType (Later will assign second column (Papers_ID) to array.)
schema = StructType([
    StructField("User", StringType(), False),
    StructField("Papers_ID", StringType(), True)
])

# Creating DataFrame df for users_libraries
userDF = sqlC.read.schema(schema)\
        .option("header", 'False').option("delimiter", ";").csv("users_libraries.txt")


#userDF.show(10, False)
userDF.printSchema()
#userDF.show(20, False)


root
 |-- User: string (nullable = true)
 |-- Papers_ID: string (nullable = true)



In [11]:
# Creating Papers.csv DataFrame
# I didnt create schema for this DF, because at the end i needed two columns and wanted to change
# type of only paper_id column to IntegerType()
paperDF = sqlC.read.csv("papers.csv")
columns= ["paper_id", "type", "journal", "bookـtitle", "series", "publisher", "pages", "volume", "number", "year", "month", "postedat","address","title", "abstract"]
oldCols = paperDF.schema.names
for i, x in enumerate(oldCols):
    paperDF = paperDF.withColumnRenamed(x, columns[i])
paperDF.show(2)
paperDF.printSchema()

+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------------------+
|paper_id|   type|             journal|bookـtitle|series|publisher|pages|volume|number|year|month|           postedat|address|               title|            abstract|
+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------------------+
|   80546|article|biology and philo...|      null|  null|     null|   17|    19|     2|2004|  mar|2005-01-26 21:35:21|   null|the arbitrariness...|the genetic code ...|
| 5842862|article|      molecular cell|      null|  null| elsevier|    2|    35|     6|2009|  sep|2009-09-30 17:11:23|   null|how to choose a g...|choosing good pro...|
+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------

# Exercise 1.3 with DataFrames ( Joining Collections )

In [12]:
#Type casting from str to array so i can apply explode function on papers_id column.
def str_to_arr(my_list):
    my_list = my_list.split(",")
    return [x for x in my_list]

str_to_arr_udf = udf(str_to_arr,ArrayType(StringType()))

userDF = userDF.withColumn('PapersID',str_to_arr_udf(userDF["Papers_ID"]))
userDF = userDF.drop("Papers_ID")
userDF.show(2, False)
userDF.printSchema()

+--------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|User                            |PapersID                                                                                                                                                                                                                                                                                                                                                                                                       |
+--------------------------------+------------------------------------------------------------------------------------------------

In [13]:
# Checking if explode function is working properly
userDF2 = userDF.select(userDF.User, sf.explode(userDF.PapersID))
userDF2 = userDF2.withColumn("Paper_ID", userDF2["col"])
userDF2 = userDF2.drop("col")
userDF2.show(2, False)

+--------------------------------+--------+
|User                            |Paper_ID|
+--------------------------------+--------+
|28d3f81251d94b09735497477a5e4e02|3929762 |
|28d3f81251d94b09735497477a5e4e02|503574  |
+--------------------------------+--------+
only showing top 2 rows



In [14]:
# Changing datatype of paper_id to IntegerType()
paperDF = paperDF.withColumn("paper_id", col("paper_id").cast(IntegerType()))
paperDF.printSchema()

root
 |-- paper_id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- bookـtitle: string (nullable = true)
 |-- series: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- pages: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- number: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- postedat: string (nullable = true)
 |-- address: string (nullable = true)
 |-- title: string (nullable = true)
 |-- abstract: string (nullable = true)



In [16]:
#Taking subset of Dataset before joining with userDF.
papersDF = paperDF["paper_id", "abstract"]
papersDF.show(2)

+--------+--------------------+
|paper_id|            abstract|
+--------+--------------------+
|   80546|the genetic code ...|
| 5842862|choosing good pro...|
+--------+--------------------+
only showing top 2 rows



In [17]:
joined = userDF2.join(papersDF, [papersDF.paper_id == userDF2.Paper_ID], "inner")
joined.show(2, False)

+--------------------------------+--------+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
# Dropping PaperID columns as we just need to count words of a user.
user_abs = joined.drop("paper_id", "Paper_ID")
user_abs.show(1, False)

+--------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Exercise 1.4 with DataFrames ( Basic Analysis )

In [19]:
#Number of Distinct Users.
dist_Users = user_abs.select(user_abs.User).distinct().count()
print("Distinct Users:",dist_Users)

#Distinct Users: 28416

dist_items = papersDF.select(papersDF.paper_id).distinct().count()
print("Distinct Items: ",dist_items)

# Distinct Items: 172079

#Total Ratings:
total_ratings = user_abs.select(user_abs.abstract).count()
print("Total Ratings: ", total_ratings)

#Total Ratings: 828481

Distinct Users: 28416
Distinct Items:  172079
Total Ratings:  828481


In [20]:
# Minimum number of Ratings a user has given.
rat_user = userDF.select(userDF.PapersID).withColumn("Count", size(userDF.PapersID))

#Minimum
rat_user.agg({"Count": 'min'}).show()

#Maximum
rat_user.agg({"Count": 'max'}).show()

#Average
rat_user.agg({"Count": 'avg'}).show()

#Std
rat_user.agg({"Count": 'std'}).show()


+----------+
|min(Count)|
+----------+
|         1|
+----------+

+----------+
|max(Count)|
+----------+
|      1922|
+----------+

+------------------+
|        avg(Count)|
+------------------+
|29.155440596846848|
+------------------+

+-----------------+
|    stddev(Count)|
+-----------------+
|81.17660451011595|
+-----------------+



In [21]:
# Minimum rating an Item has received
#min_rat_item = userDF2.select(userDF2.Paper_ID).withColumn("Count", )

min_rat_items = userDF2.groupBy("Paper_ID").count()

#Minimum
min_rat_items.agg({"Count": 'min'}).show()

#Maximum
min_rat_items.agg({"Count": 'max'}).show()

#Average
min_rat_items.agg({"Count": 'avg'}).show()

# Standard Deviation
min_rat_items.agg({"Count": "std"}).show()
#5.477818208917285

+----------+
|min(Count)|
+----------+
|         3|
+----------+

+----------+
|max(Count)|
+----------+
|       924|
+----------+

+----------------+
|      avg(Count)|
+----------------+
|4.81453867119172|
+----------------+

+-----------------+
|    stddev(Count)|
+-----------------+
|5.477818208917285|
+-----------------+



In [4]:
tryy = userRatingsRDD.take(10) 

In [5]:
tryy

[('f05bcffe7951de9e5a32fff4a42eb088',
  ['1158654',
   '478707',
   '12054725',
   '6670515',
   '781057',
   '13329754',
   '781055',
   '2825',
   '553840',
   '988447',
   '12938139',
   '12037219',
   '12788583',
   '6595566',
   '4027225',
   '920055',
   '3129258',
   '2242776',
   '3112352',
   '144287',
   '706033',
   '525396',
   '622633',
   '9172127',
   '7357993',
   '230211',
   '12790816',
   '503161',
   '12937120',
   '942241',
   '166220',
   '8493138',
   '2945819',
   '227173',
   '11191048',
   '949352',
   '227096',
   '11597926',
   '921623',
   '833638',
   '882809',
   '12738996',
   '99',
   '7154210',
   '11852474',
   '72879',
   '10723139',
   '11923443',
   '2862386',
   '802634',
   '1959297',
   '4488840',
   '12735311',
   '5102465',
   '5177766',
   '129',
   '1467354',
   '9396734',
   '494336',
   '1218954',
   '2783692',
   '3733678',
   '8423325',
   '1160828',
   '507529',
   '7804574',
   '5201306',
   '407124',
   '504896',
   '3281478',
   '136

In [7]:
t = sqlC.createDataFrame(tryy)

In [8]:
t.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|f05bcffe7951de9e5...|[1158654, 478707,...|
|28d3f81251d94b097...|[3929762, 503574,...|
|d0c9aaa788153daea...|[2080631, 6343346...|
|ca4f1ba4094011d9a...|            [278019]|
|d1d41a15201915503...|[6610569, 6493797...|
|f2f77383828ea6d39...|[943458, 238121, ...|
|9c883d02115400f7b...|[3509971, 3509965...|
|b656009a6efdc8b1a...|[771870, 181369, ...|
|cf9c7f356092c34be...|             [90558]|
|0f5cbb39410a9278f...|           [9344598]|
+--------------------+--------------------+

